## AuxTel azimuth oscillations

Craig Lage 30-Aug-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsSeqNumToVisitId
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all"])

client = EfdClient('usdf_efd')

In [ ]:
start = Time("2024-01-10T00:00:00", scale='utc')
end = Time("2024-01-10T01:00:00", scale='utc')


In [ ]:
seqNums = [61, 90, 94, 95, 96, 100, 101]
dayObs = 20240827
expIds = []
azs = []
els = []
az1_spds = []
az2_spds = []
for seqNum in seqNums:
    expId = dayObsSeqNumToVisitId(dayObs, seqNum)
    expIds.append(expId)
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    azs.append(mData['AZSTART'])
    els.append(mData['ELSTART'])
    start = Time(mData['DATE-BEG'], format='isot', scale='tai').utc
    end = Time(mData['DATE-END'], format='isot', scale='tai').utc
    mount_speed = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'],
                                              start, end)
    az1_spds.append(np.median(mount_speed['azimuthMotor1Velocity'].values) * 3600.0)
    az2_spds.append(np.median(mount_speed['azimuthMotor2Velocity'].values) * 3600.0)

In [ ]:
print("expId             Az       El        AzMotor1Spd     AzMotor2Spd  ")
for i in range(len(expIds)):
    print(f"{expIds[i]}     {azs[i]:.1f}     {els[i]:.1f}      {az1_spds[i]:.2f}             {az2_spds[i]:.2f}")